## Deployment Infraestructure

Once the model is packaged, it needs to be deployed using an appropriate infrastructure. Deployment involves making the model accessible for predictions in a production environment, ensuring scalability, reliability, and maintainability. Below are the key considerations and steps for deploying a machine learning model.

1) Deployment pattern: 
    - Batch Processing: Best for periodic predictions (e.g., fraud detection, risk analysis), normally used scheduled jobs or pipelines that run the model periodically.
        - Data Processing Pipelines: Using Apache Airflow, Prefect, or Kubeflow.
        - ETL/ELT Jobs: Scheduled jobs using AWS Glue, Azure Data Factory, or custom scripts.
    - Real-Time Inference: For applications needing instant responses (e.g., recommendation systems, chatbots). 
        - REST APIs: Using Flask, FastAPI, Django REST Framework.
        - gRPC its more eficient but complex.Also good for web aplications
    - Edge Deployment : Running models on IoT or mobile devices (e.g., image recognition on phones). Might use lightweight APIs, SDKs, or specialized protocols

2) Environment management: how the model will run consistently across different environment.
    - Virtual Environments: Use venv or conda to manage dependencies in different environments. 
    - Containerization
        1) Docker: Enables portability, reproducibility and isolated environments. Packages model, dependencies, and API into a lightweight container.
        2) Kubernetes: A container orchestration platform designed for scaling deployments in cloud environments. It efficiently manages multiple Docker containers, automating deployment, scaling, and operations across distributed infrastructure. So it can manage several model instances.


3) Infraestructure selection: define how the model will be accessed 
    - AWS: Deploy via SageMaker, Lambda, ECS/Fargate, S3 (batch), API Gateway.
    - Azure: Use Azure ML, Azure Functions, AKS (Kubernetes), Azure Batch.
    - GCP: Deploy with Vertex AI, Cloud Functions, AI Platform, Cloud Run.
    - On-Premise / Hybrid: Use Kubernetes (K8s), OpenShift, MLflow for flexible deployments.



4) Automation & DevOps: ensure the model deployment process is efficient, reproducible, and scalable across various environments. This involves two key components:
    - Infrastructure as Code (IaC): Automates the provisioning and management of infrastructure, ensuring reproducibility and scalability.Implementation varies by deployment pattern:
        - Batch Processing: Use IaC tools like Terraform, AWS CloudFormation, or Azure Bicep to automate the setup of batch processing pipelines (AWS Batch, Azure Batch).
        - Cloud Platforms: Use IaC tools (e.g., Terraform, CloudFormation) to automate the provisioning of cloud resources (e.g., SageMaker endpoints, Lambda functions), maintaining consistent configurations across environments.
        - On-Premise/Hybrid: Implement IaC to manage Kubernetes clusters for containerized model deployment, ensuring consistent infrastructure across private and public clouds.
    - CI/CD for ML Deployment:Automates the testing, building, and deployment of models using GitHub Actions, GitLab CI/CD, or Jenkins.Implement monitoring and rollback strategies for failed deployments.Normally Real-Time Inference use CI/CD pipelines for deployment of API-based models.

 

Relationships Between Components

- Deployment Pattern → Environment → Infrastructure: Your chosen deployment pattern influences your environment management needs, which in turn determines the most appropriate infrastructure.

- Environment Management → API Development: Your environment management strategy affects how you develop and deploy APIs. For example, containerized environments require different API deployment approaches than virtual environments.

- Infrastructure → Automation: Different infrastructure providers require different automation tools. AWS deployments might use CloudFormation, while multi-cloud deployments benefit from Terraform.


More common Deployments:


| Entorno  | Enfoque  |
|:------|:------:|
| On-Premise | Docker, Kubernetes (on-premise), Virtual Environments (venv, conda), Batch Processing, gRPC APIs, Fast API |
| Hybrid | Kubernetes (AKS, GKE, EKS), Cloud Storage + On-Prem Compute, Edge Deployment, Microservices. |
| Cloud	 | Serverless low latence (AWS Lambda, Azure Functions), Cloud ML Services if the model is heavy (SageMaker, Vertex AI, Azure ML), REST APIs. |





### Rest API

#### Flask API
Synchronous: Handles requests one at a time (blocking I/O).

Advantages: Simpler: Good for basic APIs and small-scale applications.k Use it if you need a quick prototype or a simple API.

Disadvantages: Requires manual request validation: You need to handle data validation yourself.Older framework: More widely used in legacy projects.Slower: Compared to FastAPI, especially under high loads.


In [ ]:
##Create a Flask API for the model 

from flask import Flask, request, jsonify
import joblib
import numpy as np

app = Flask(__name__)

# Load the trained model
model = joblib.load('random_forest.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    prediction = model.predict(np.array(data["features"]).reshape(1, -1))
    return jsonify({'prediction': int(prediction[0])})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)



#### FastAPI API
Asynchronous (ASGI): Can handle multiple requests concurrently (non-blocking I/O).

Advantages:Automatic validation: Uses Pydantic for input validation. Much faster: Thanks to Starlette (underlying framework).Built-in Swagger and Redoc: Automatically generates API documentation. Modern and recommended: Ideal for production-level machine learning APIs.

In [ ]:
from fastapi import FastAPI
import pickle
import pandas as pd

app = FastAPI()

# Load the pretained model
with open("modelo_fraude.pkl", "rb") as file:
    modelo = pickle.load(file)

@app.post("/predecir")
def predecir(datos: dict):
    df = pd.DataFrame([datos])
    prediccion = modelo.predict(df)
    return {"fraude": bool(prediccion[0])}


### CI/CD for ML Deployment

#### GitHub Actions
Use it for: 
- Train the model
- Packged on docker Docker.
- Uploaded to he cloud.

In [ ]:
name: Deploy Model

on: push

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout code
        uses: actions/checkout@v2

      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.9'

      - name: Install dependencies
        run: pip install -r requirements.txt

      - name: Train the Model
        run: python train.py

      - name: Build Docker image
        run: docker build -t mi-api-fraude .

      - name: Push Docker image to AWS
        run: docker push mi-repo-docker/mi-api-fraude

      - name: Deploy to AWS Lambda
        run: aws lambda update-function-code --function-name mi-api --image-uri mi-repo-docker/mi-api-fraude
